In [1]:
# 필요한 패키지 불러오기
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from gensim.models import FastText
from sklearn.model_selection import train_test_split

# 데이터 로드
train_df = pd.read_csv('train_for_NLP.csv')
test_df = pd.read_csv('test_cleaned.csv')

# train 데이터에서 특성과 레이블 분리
X_train = train_df['combined_str']
y_train = train_df['target']

# test 데이터의 특성
X_test = test_df['combined_str']

# 문장을 단어 리스트로 분리
sentences = [text.split() for text in X_train]

# FastText 모델 학습 (Skipgram 사용)
fasttext_model = FastText(sentences, vector_size=100, window=5, sg=1, workers=4)

# FastText로 문서 벡터화
def vectorize_fasttext(text, model):
    words = text.split()
    # 각 문서 내 단어 벡터의 평균 계산
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    
    # 단어 벡터가 없는 경우에는 0으로 채운 벡터 반환
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# FastText로 변환된 데이터를 고정된 크기의 벡터로 변환
X_train_fasttext = np.array([vectorize_fasttext(text, fasttext_model) for text in X_train])
X_test_fasttext = np.array([vectorize_fasttext(text, fasttext_model) for text in X_test])

# 스케일링
scaler = StandardScaler()
X_train_fasttext_scaled = scaler.fit_transform(X_train_fasttext)
X_test_fasttext_scaled = scaler.transform(X_test_fasttext)

# 1D CNN 모델 구축
def create_cnn_model(input_shape):
    model = tf.keras.models.Sequential()

    # 1D Convolutional Layer
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    
    # MaxPooling Layer
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
    
    # Flatten Layer
    model.add(tf.keras.layers.Flatten())
    
    # Dense Layer
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    
    # Output Layer (이진 분류)
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile Model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# 데이터를 학습과 검증용으로 분리
X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(X_train_fasttext_scaled, y_train, test_size=0.2, random_state=42)

# 1D CNN 모델 정의
input_shape = (X_train_cnn.shape[1], 1)
cnn_model = create_cnn_model(input_shape)

# 데이터를 CNN의 입력 형식으로 변환
X_train_cnn = np.expand_dims(X_train_cnn, axis=-1)
X_val_cnn = np.expand_dims(X_val_cnn, axis=-1)
X_test_cnn = np.expand_dims(X_test_fasttext_scaled, axis=-1)

# 모델 학습
history = cnn_model.fit(X_train_cnn, y_train_cnn, validation_data=(X_val_cnn, y_val_cnn), epochs=10, batch_size=32)

# 테스트 데이터 예측
y_test_pred_prob_cnn = cnn_model.predict(X_test_cnn).flatten()
y_test_pred_cnn = (y_test_pred_prob_cnn >= 0.5).astype(int)

# 제출 파일 생성
submission_df = pd.read_csv("sample_submission.csv")
submission_df['target'] = y_test_pred_cnn
submission_df.to_csv("submit_FastText_1DCNN.csv", index=False)


Epoch 1/10
191/191 [==============================] - 3s 8ms/step - loss: 0.5988 - accuracy: 0.6867 - val_loss: 0.5681 - val_accuracy: 0.7196
Epoch 2/10
191/191 [==============================] - 1s 6ms/step - loss: 0.5653 - accuracy: 0.7113 - val_loss: 0.5638 - val_accuracy: 0.7301
Epoch 3/10
191/191 [==============================] - 1s 6ms/step - loss: 0.5565 - accuracy: 0.7212 - val_loss: 0.5547 - val_accuracy: 0.7308
Epoch 4/10
191/191 [==============================] - 1s 6ms/step - loss: 0.5440 - accuracy: 0.7218 - val_loss: 0.5602 - val_accuracy: 0.7098
Epoch 5/10
191/191 [==============================] - 1s 6ms/step - loss: 0.5384 - accuracy: 0.7363 - val_loss: 0.5542 - val_accuracy: 0.7255
Epoch 6/10
191/191 [==============================] - 1s 6ms/step - loss: 0.5351 - accuracy: 0.7325 - val_loss: 0.5634 - val_accuracy: 0.7209
Epoch 7/10
191/191 [==============================] - 1s 6ms/step - loss: 0.5331 - accuracy: 0.7392 - val_loss: 0.5505 - val_accuracy: 0.7295
Epoch 